# Exercice : Bert & Systèmes de Questions/Réponses

<img src="./qa_header.png" width="1000">

On va utiliser la librairie [`transformers`](https://huggingface.co/transformers/index.html) qui fait référence pour le déploiement de modèles pré-entrainés.

In [28]:
import os
import torch
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
)

In [57]:
# *!pip install git+https://github.com/AntoineSimoulin/titulus

In [56]:
from titulus import color, print_

## Bert

Les architectures transformers ont inspiré le modèle Bert <span class="badge badge-secondary">Devlin et al. (2019)</span> qui consiste en une succession de transformers. Ce modèle est présent également la spécificité d’être pré-entrainé : les poids sont initialisés en entrainant le modèle sur une tâche auto-supervisée. En l’occurrence prédire des mots masqués, on parle de modèle de langue masqué. Ce modèle est très versatile et peut être décliné pour quasiment tous les cas d’usages du NLP.


<span class="badge badge-secondary">Devlin et al. (2019)</span> Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. NAACL-HLT (1) 2019: 4171-4186

In [8]:
config_name = os.path.join('camembert-qa', 'config.json')
model_name_or_path = './camembert-qa'

do_lower_case = False

In [9]:
config = AutoConfig.from_pretrained(
    config_name, 
    cache_dir=None)

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    do_lower_case=do_lower_case,
    cache_dir=None,
)

model = AutoModel.from_pretrained(
    model_name_or_path,
    from_tf=False,
    config=config,
    cache_dir=None,
)

Some weights of CamembertModel were not initialized from the model checkpoint at ./camembert-qa and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
inputs = tokenizer("Longtemps, je me suis couché de bonne heure ", return_tensors="pt")

In [23]:
inputs['input_ids'].shape

torch.Size([1, 11])

In [24]:
outputs = model(**inputs)

In [25]:
outputs[0].shape

torch.Size([1, 11, 768])

In [26]:
outputs[1].shape

torch.Size([1, 768])

## Questions/Réponses

<img src="https://fquad.illuin.tech/authors/admin/avatar_huc53b72e0c91a944ce6ad6c5a4ef38357_37416_270x270_fill_lanczos_center_2.png" width="100">

FQuAD <span class="badge badge-secondary">Hoffschmidt et al. (2020)</span> est un jeu de données pour la compréhension de lecture en français constitué de plus de 25 000 questions créées à partir d'un ensemble d'articles de Wikipédia. Il est construit sur le modèle du jeu de données anglais, SQuAD. La création du jeu de données a été supervisée par l'entreprise [Illuin Technology](https://fquad.illuin.tech/). Le jeu de données est sous license [CC BY-NC-SA 3.0](https://creativecommons.org/licenses/by-nc-sa/3.0/fr/), ce qui restreint en particulier son utilisation commerciale. 

J'ai récupéré les données et j'ai entrainé un modèle de Questions/Réponse (Q&A) à partir de l'architecture recommandée dans l'[article](https://arxiv.org/pdf/2002.06071.pdf) <span class="badge badge-secondary">Martin et al. (2020)</span>. Vous pouvez récupérer les poids directement sur le [Moodle](https://moodle.u-paris.fr/course/view.php?id=11048#section-9). Comme pour le modèle GPT-2 de la semaine dernière, j'ai réalisé l'entrainement sur un TPU dans l'environnement [Google COLAB](https://colab.research.google.com/). 

La semaine dernière, j'avais procédé au pré-entrainement complet du modèle qui avait pris environ une semaine. Cette fois-ci, je n'ai opéré que le "fine-tuning" d'un modèle déjà pré-entrainé : Camembert. Cela n'a pris que 1h sur un TPU.

<span class="badge badge-secondary">Hoffschmidt et al. (2020)</span> Martin d'Hoffschmidt, Maxime Vidal, Wacim Belblidia, Tom Brendlé: FQuAD: French Question Answering Dataset. CoRR abs/2002.06071 (2020)

<span class="badge badge-secondary">Martin et al. (2020)</span> Louis Martin, Benjamin Müller, Pedro Javier Ortiz Suárez, Yoann Dupont, Laurent Romary, Éric de la Clergerie, Djamé Seddah, Benoît Sagot: CamemBERT: a Tasty French Language Model. ACL 2020: 7203-7219

In [10]:
model_qa = AutoModelForQuestionAnswering.from_pretrained(
    model_name_or_path,
    from_tf=False,
    config=config,
    cache_dir=None,
)

Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at ./camembert-qa and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
def color_answer_in_text(question, context, answer_start, answer_end):
    offset = len(tokenizer.tokenize(question)) + 3
    tokens = tokenizer.tokenize(context)
    weights = [1 if ((i >= answer_start - offset) and (i < answer_end - offset)) 
               else 0 for i in range(len(tokens))]


    tokens_clean = []
    offset_clean = []

    
    for t, w in zip(tokens, weights):
        if t.startswith('▁'):
            offset_clean.append(w)
            tokens_clean.append(t.strip('▁'))
        else:
            tokens_clean[-1] += t
            offset_clean[-1] = max(offset_clean[-1], w)

    print_(' '.join(color(tokens_clean, offset_clean, n=10)))

In [115]:
question = "Quand est venu au monde Paul Jules Antoine Meillet ?"


context = """Paul Jules Antoine Meillet, né le 11 novembre 1866 à Moulins (Allier) et mort 
le 21 septembre 1936 à Châteaumeillant (Cher), est le principal linguiste français des 
premières décennies du xxe siècle. Il est aussi philologue. D'origine bourbonnaise, fils 
d'un notaire de Châteaumeillant (Cher), Antoine Meillet fait ses études secondaires au lycée 
de Moulins. Étudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment 
les cours de Louis Havet, il assiste également à ceux de Michel Bréal au Collège de France et 
de Ferdinand de Saussure à l'École pratique des hautes études."""


# 1. On tokenize l'input
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. On prédit avec le modèle l'index de début et de fin de la réponse dans le texte
answer_start_scores, answer_end_scores = model_qa(**inputs)
answer_start = torch.argmax(answer_start_scores)  
answer_end = torch.argmax(answer_end_scores) + 1  

# 3. On renvoie le span de texte avec la réponse
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'11 novembre 1866'

In [116]:
color_answer_in_text(question, context, answer_start, answer_end)

In [113]:
question = "Quel est le métier de Paul Jules Antoine Meillet ?"


context = """Paul Jules Antoine Meillet, né le 11 novembre 1866 à Moulins (Allier) et mort 
le 21 septembre 1936 à Châteaumeillant (Cher), est le principal linguiste français des 
premières décennies du xxe siècle. Il est aussi philologue. D'origine bourbonnaise, fils 
d'un notaire de Châteaumeillant (Cher), Antoine Meillet fait ses études secondaires au lycée 
de Moulins. Étudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment 
les cours de Louis Havet, il assiste également à ceux de Michel Bréal au Collège de France et 
de Ferdinand de Saussure à l'École pratique des hautes études."""


# 1. On tokenize l'input
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. On prédit avec le modèle l'index de début et de fin de la réponse dans le texte
answer_start_scores, answer_end_scores = model_qa(**inputs)
answer_start = torch.argmax(answer_start_scores)  
answer_end = torch.argmax(answer_end_scores) + 1  

# 3. On renvoie le span de texte avec la réponse
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'linguiste français'

In [114]:
color_answer_in_text(question, context, answer_start, answer_end)

In [108]:
question = "Quel est le vrai nom de Jul ?"


context = """Jul, de son vrai nom Julien Mari1, est un rappeur et chanteur français, 
né le 14 janvier 1990 dans le 12e arrondissement de Marseille.
Il publie son premier single, Sort le cross volé, en novembre 2013 suivi en février 2014 
d'un album entier, Dans ma paranoïa, le premier d'une série prolifique : 
deux albums complets par an depuis le début de sa carrière, tous certifiés au moins disque de platine.
En 2015, Jul quitte le label Liga One Industry à la suite de désaccords financiers et 
fonde son propre label indépendant, D'or et de platine. L'année suivante, il reçoit la récompense 
du meilleur album de musique urbaine aux 32es Victoires de la musique pour l'album My World."""


# 1. On tokenize l'input
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. On prédit avec le modèle l'index de début et de fin de la réponse dans le texte
answer_start_scores, answer_end_scores = model_qa(**inputs)
answer_start = torch.argmax(answer_start_scores) 
answer_end = torch.argmax(answer_end_scores) + 1 

# 3. On renvoie le span de texte avec la réponse
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'Julien Mari1,'

In [109]:
color_answer_in_text(question, context, answer_start, answer_end)